# Tensorflow Example

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import mlflow
from dotenv import load_dotenv
import os, datetime
import boto3

In [2]:
# Settting up environment variables

load_dotenv()  

S3_BUCKET = os.getenv("S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
TRACKING_URI = os.getenv("TRACKING_URI")

In [3]:
boto3.setup_default_session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

# Prearing the dataset

In [4]:
(ds_train, ds_test), ds_info = tfds.load(
 "mnist",
 split=["train", "test"],
 shuffle_files=True,
 as_supervised=True,
 with_info=True,
)
# Normalizes images: `uint8` -> `float32`
def normalize_img(image, label):
 return tf.cast(image, tf.float32) / 255., label
 
# Train Dataset
ds_train = ds_train.map(
 normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
# Test Dataset
ds_test = ds_test.map(
 normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# Defining the model

In [16]:
# Define the layers
inputs = tf.keras.Input(shape=(28, 28, 1))
hidden = tf.keras.layers.Flatten()(inputs)
hidden2 = tf.keras.layers.Dense(64, activation="relu")(hidden)
outputs = tf.keras.layers.Dense(10, activation="softmax")(hidden2)
# Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.002)
# Create a Model object
model = tf.keras.Model(inputs, outputs)
# Compile the model
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [17]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                50240     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


In [23]:
BATCH_SIZE = 128
EPOCHS = 20
LOGGING_STEPS = 10

MODELNAME = "Simple_Net"
EXPERIMENT = "MNIST"
RUN_NAME = "Run_8"

logdir = os.path.join("logs", MODELNAME, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))


In [24]:
logdir

'logs\\Simple_Net\\20221003-194940'

In [25]:
mlflow.tensorflow.autolog(every_n_iter=LOGGING_STEPS)
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT)
mlflow.start_run(run_name=RUN_NAME)

<ActiveRun: >

In [26]:
# Tensorflow Callbacks

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(MODELNAME, save_weights_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                min_delta=0, 
                                patience=4, 
                                verbose=0, 
                                mode='auto', 
                                restore_best_weights=False)

In [28]:
history = model.fit(ds_train, epochs=EPOCHS, validation_data=ds_test,
    batch_size=BATCH_SIZE,
    use_multiprocessing = True,
    callbacks = [tensorboard_callback, checkpoint, early_stopping])
mlflow.end_run()

2022/10/03 19:50:46 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'bd5cf2f1a5624becb30020055b86e6f5', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/20
  1/469 [..............................] - ETA: 3:21 - loss: 0.0079 - accuracy: 1.0000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0029s vs `on_train_batch_end` time: 0.0651s). Check your callbacks.


469/469 [==============================] - 4s 7ms/step - loss: 0.0156 - accuracy: 0.9955 - val_loss: 0.0941 - val_accuracy: 0.9744
Epoch 2/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0148 - accuracy: 0.9955 - val_loss: 0.0946 - val_accuracy: 0.9741
Epoch 3/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0128 - accuracy: 0.9966 - val_loss: 0.0978 - val_accuracy: 0.9742
Epoch 4/20
469/469 [==============================] - 4s 8ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.0960 - val_accuracy: 0.9757
Epoch 5/20
469/469 [==============================] - 3s 6ms/step - loss: 0.0103 - accuracy: 0.9970 - val_loss: 0.1040 - val_accuracy: 0.9766
INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmp80ud2dr4\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmp80ud2dr4\model\data\model\assets
